# Black Scholes Exercise 2: NumPy implementation

- Use cProfile and Line Profiler to look for bottlenecks and hotspots in the code

In [1]:
# Boilerplate for the example

import cProfile
import pstats
import numpy as np
%load_ext line_profiler

try:
    import numpy.random_intel as rnd
except:
    import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range

SEED = 7777777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

nopt=100000
price, strike, t = gen_data(nopt)
call = np.zeros(nopt, dtype=np.float64)
put  = -np.ones(nopt, dtype=np.float64)

# The NumPy modified naive Black Scholes algorithm (looped)

- Minimally converted code from the Naive example
- TODO: Convert the math import to numpy variants

In [2]:
import numpy as np
from math import log, sqrt, exp, erf
invsqrt = lambda x: 1.0/sqrt(x)

def black_scholes ( nopt, price, strike, t, rate, vol, call, put ):
    mr = -rate
    sig_sig_two = vol * vol * 2
    
    for i in range(nopt):
        P = price[i]
        S = strike[i]
        T = t[i]
        
        a = log(P / S)
        b = T * mr
        
        z = T * sig_sig_two
        c = 0.25 * z
        y = invsqrt(z)
        
        w1 = (a - b + c) * y
        w2 = (a - b - c) * y
        
        d1 = 0.5 + 0.5 * erf(w1)
        d2 = 0.5 + 0.5 * erf(w2)
        
        Se = exp(b) * S
        
        call[i] = P * d1 - Se * d2
        put[i] = call[i] - P + Se
        #print(call,put)


## Run timeit, cProfile, and line_profiler to see what is happening

In [3]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)

342 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
cProfile.run('black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)')

         600004 function calls in 0.481 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100000    0.028    0.000    0.041    0.000 <ipython-input-2-b186b5e5aa49>:3(<lambda>)
        1    0.335    0.335    0.481    0.481 <ipython-input-2-b186b5e5aa49>:5(black_scholes)
        1    0.000    0.000    0.481    0.481 <string>:1(<module>)
        1    0.000    0.000    0.481    0.481 {built-in method builtins.exec}
   200000    0.070    0.000    0.070    0.000 {built-in method math.erf}
   100000    0.014    0.000    0.014    0.000 {built-in method math.exp}
   100000    0.021    0.000    0.021    0.000 {built-in method math.log}
   100000    0.013    0.000    0.013    0.000 {built-in method math.sqrt}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [5]:
%lprun -f black_scholes black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)

## _Why does this example take longer, even with performance libraries?_

When using NumPy, we need to unloop the function and utilize NumPy Arrays to achieve vectorization.  
Notice the Below function removes the loop, but now passes in the strike, price, and t as an _entire_ array

In [6]:
from numpy import log, exp, erf, invsqrt
def black_scholes (nopt, price, strike, t, rate, vol ):
    mr = -rate
    sig_sig_two = vol * vol * 2

    P = price
    S = strike
    T = t

    a = log(P / S)
    b = T * mr

    z = T * sig_sig_two
    c = 0.25 * z
    y = invsqrt(z)

    w1 = (a - b + c) * y
    w2 = (a - b - c) * y

    d1 = 0.5 + 0.5 * erf(w1)
    d2 = 0.5 + 0.5 * erf(w2)

    Se = exp(b) * S

    call = P * d1 - Se * d2
    put = call - P + Se
    
    return (call, put)

## Run timeit, cProfile, and line_profiler to see what is happening

In [7]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

3.33 ms ± 86.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
cProfile.run('black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)')

         4 function calls in 0.005 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.005    0.005    0.005    0.005 <ipython-input-6-f4ecbfbbabfd>:2(black_scholes)
        1    0.000    0.000    0.005    0.005 <string>:1(<module>)
        1    0.000    0.000    0.005    0.005 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [9]:
%lprun -f black_scholes black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)